In [1]:
# In[1]:
# import subprocess
# subprocess.call(['sh', './install_packages.sh'])

import taichi as ti
import numpy as np
import math
from sympy import inverse_mellin_transform
from pyevtk.hl import gridToVTK
import pandas as pd
import matplotlib.pyplot as plt
import os
import argparse
import random
import time
import tifffile
import imagecodecs


plt.rcParams['font.family'] = 'DeJavu Serif' 
plt.rcParams["font.serif"] = "Times New Roman"

ti.init(arch=ti.cpu)


dim = 3
Q = 19 
real = ti.f32
i32_vec3d = ti.types.vector(dim, ti.i32)
f32_vec3d = ti.types.vector(dim, ti.f32)
scalar_real = lambda: ti.field(dtype=real)
scalar_int = lambda: ti.field(dtype=ti.i32)
vec = lambda: ti.Vector.field(dim, dtype=real)
vec_Q = lambda: ti.Vector.field(Q, dtype=real)



# Input paramters
lx = 200

ly = lz = 100


IniPerturbRate = 1
rho0 = 1
rhos = 3
carn_star = False
T_Tc = 0.7
G = -1.0
tau =1.  # specify the relaxaton time (only for BGK operator)
inv_tau = 1/tau
A = tau

dp = scalar_real() # pressure at left
loss = scalar_real()
sum_v = scalar_real()
sum_rho = scalar_real()

is_solid = scalar_int()

v = vec()

temp_v = vec()
target_v =  vec() # target velocity field
rho = scalar_real()

collide_f = vec_Q()
stream_f = vec_Q()

ti.root.dense(ti.ijk, (lx, ly, lz)).place(is_solid)
ti.root.dense(ti.ijk, (lx, ly, lz)).place(target_v)
ti.root.dense(ti.ijk, (lx, ly, lz)).place(rho,v,temp_v)
ti.root.dense(ti.ijk, (lx, ly, lz)).place(collide_f,stream_f)

ti.root.place(dp, loss,sum_v,sum_rho)

ti.root.lazy_grad()

"""Definition of LBM parameters"""
half = (Q - 1) // 2
# LBM weights
t0 = 1.0 / 3.0
t1 = 1.0 / 18.0
t2 = 1.0 / 36.0
w_np = np.array(
    [t0, t1, t1, t1, t1, t1, t1, t2, t2, t2, t2, t2, t2, t2, t2, t2, t2, t2, t2]
)

# x component of predefined velocity in Q directions
e_xyz_list = [
    [0, 0, 0],
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1],
    [1, 1, 0],
    [-1, -1, 0],
    [1, -1, 0],
    [-1, 1, 0],
    [1, 0, 1],
    [-1, 0, -1],
    [1, 0, -1],
    [-1, 0, 1],
    [0, 1, 1],
    [0, -1, -1],
    [0, 1, -1],
    [0, -1, 1],
]

# reversed_e_xy_np stores the index of the opposite component to every component in e_xy_np
# For example, for [1,0,0], the opposite component is [-1,0,0] which has the index of 2 in e_xy
reversed_e_index = np.array([e_xyz_list.index([-a for a in e]) for e in e_xyz_list])

w = ti.field(ti.f32,shape=Q)
w.from_numpy(w_np)
e_xyz = ti.Vector.field(n=dim, dtype=ti.i32, shape=Q)
e_xyz.from_numpy(np.array(e_xyz_list))
ti.static(e_xyz)
ti.static(w)

os.chdir('./')


# In[2]:


# pip install tifffile
# pip3 install imagecodecs

# ## Read TIFF images

# In[3]:

tiff = "./Sand_002_00_labelled.tif"

print("Reading trinarized image Sand_002_00_labelled.tif.")
trinarized_00 = tifffile.imread(tiff)
print("Done.")

scale_factor = 3
xmin = int(trinarized_00.shape[0]/2-lx/2*scale_factor)
ymin = int(trinarized_00.shape[1]/2-ly/2*scale_factor)
zmin = int(trinarized_00.shape[2]/2-lz/2*scale_factor)
xmax = xmin + lx*scale_factor
ymax = ymin + ly*scale_factor
zmax = zmin + lz*scale_factor

print(xmin,xmax,ymin,ymax,zmin,zmax)

grains = trinarized_00[xmin:xmax:scale_factor,ymin:ymax:scale_factor,zmin:zmax:scale_factor]
grain_index = np.unique(grains)

print(grains.shape,grain_index.size)
solid_3darray = np.sign(grains)

is_solid.from_numpy(solid_3darray)

for i in range(ly):
    for j in range(lz):
        is_solid[0,i,j] = 0
        is_solid[lx-1,i,j] = 0
        is_solid[i,j,0]=1
        is_solid[i,j,ly-1]=1

ti.static(is_solid)



[Taichi] version 1.1.3, llvm 10.0.0, commit 1262a70a, linux, python 3.10.9


Error: Can not import avx core while this file exists: /home/amber/.local/lib/python3.10/site-packages/paddle/fluid/core_avx.so


[I 02/09/23 16:09:16.009 6445] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64
Reading trinarized image Sand_002_00_labelled.tif.
Done.
100 700 250 550 250 550
(200, 100, 100) 444


<ti.field>

In [2]:
# solid_count =  np.count_nonzero(solid_3darray)
# porosity = (solid_3darray.size - solid_count)/solid_3darray.size
solid_count = len(is_solid.to_numpy()[1:lx-1,:,:][np.nonzero(is_solid.to_numpy()[1:lx-1,:,:])])
porosity = 1-solid_count/(lx-2)/ly/lz
K = 1/(lx*ly*lz-solid_count)
print(solid_count,porosity)

def dPress(rho_value,carn_star=False):
    if carn_star:
        a = 1.0
        b = 4.0
        R = 1.0
        Tc = 0.0943
        T = T_Tc * Tc
        eta = b * rho_value / 4.0
        eta2 = eta * eta
        eta3 = eta2 * eta
        rho2 = rho_value * rho_value
        one_minus_eta = 1.0 - eta
        one_minus_eta3 = one_minus_eta * one_minus_eta * one_minus_eta

        return rho_value * R * T * (1 + eta + eta2 - eta3) / one_minus_eta3 - a * rho2
    else:
        cs2 = 1.0 / 3.0
        psi = 1.0 - np.exp(-rho_value)
        psi2 = psi * psi
        return cs2 * rho_value + cs2 * G / 2 * psi2
    
@ti.func
def periodic_index(i):
    iout = i
    if i[0] < 0:
        iout[0] = lx - 1
    if i[0] > lx - 1:
        iout[0] = 0
    if i[1] < 0:
        iout[1] = ly - 1
    if i[1] > ly - 1:
        iout[1] = 0
    if i[2] < 0:
        iout[2] = lz - 1
    if i[2] > lz - 1:
        iout[2] = 0
    return iout

@ti.func
def velocity_vec(local_pos) -> f32_vec3d:
    velocity_vec = ti.Vector([0.0, 0.0, 0.0])
    for i in ti.static(range(3)):
        for s in ti.static(range(Q)):
            velocity_vec[i] = velocity_vec[i] + stream_f[local_pos][s] * e_xyz[s][i]
        velocity_vec[i] = velocity_vec[i] / rho[local_pos]
    return velocity_vec

@ti.func
def feq_p(k,rho_local, u): # anti bounce-back pressue bound
    eu = e_xyz[k].dot(u)
    uv = u.dot(u)
    feqout = 2*w[k]*rho_local*(1.0+4.5*eu*eu-1.5*uv)
    return feqout

@ti.kernel
def init_field():
    for x,y,z in ti.ndrange(lx, ly, lz):
        rho[x,y,z] = rhos
        v[x,y,z] = ti.Vector([0., 0., 0.])
        collide_f[x,y,z] = ti.Vector([
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
            ])
        stream_f[x,y,z] = ti.Vector([
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
            ])

        if is_solid[x,y,z] <= 0:
            rho[x,y,z] = rho0
            for q in ti.static(range(Q)):
                collide_f[x,y,z][q] = w[q] * rho[x,y,z]
                stream_f[x,y,z][q] = w[q] * rho[x,y,z]      

@ti.kernel
def collision():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and I.z < lz and is_solid[I] <= 0):            
            """BGK operator"""
            v[I] = velocity_vec(I)
            u_squ = v[I].dot(v[I])
            for s in ti.static(range(Q)):
                eu = e_xyz[s].dot(v[I])
                collide_f[I][s] = collide_f[I][s] + (w[s] * rho[I] *(1.0 + 3.0 * eu + 4.5 * eu * eu - 1.5 * u_squ) -collide_f[I][s]) *inv_tau

@ti.kernel
def post_collsion():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and I.z < lz and is_solid[I] <= 0):
            collide_f[I] = stream_f[I]
            rho[I] = collide_f[I].sum()

@ti.kernel
def boundary_condition():   
    for I in ti.grouped(v):
        if (I.x < lx and I.y<ly and I.z < lz and is_solid[I]<=0):
            for s in ti.static(range(Q)):
                neighbor_pos = periodic_index(I+e_xyz[s])
                if (e_xyz[s][0] == 1 and I.x==0):
                    stream_f[I][s] = feq_p(s, dp[None], v[I])- collide_f[I][reversed_e_index[s]]

                if (e_xyz[s][0] == -1 and I.x==lx-1):
                    stream_f[I][s]= feq_p(s, rho0, v[I])- collide_f[I][reversed_e_index[s]]
    
    for I in ti.grouped(v):
        if (I.x < lx and I.y<ly and I.z < lz and is_solid[I] <= 0): 
            rho[I] = stream_f[I].sum()
            collide_f[I] = stream_f[I] 

@ti.kernel
def streaming():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and I.z < lz and is_solid[I] <= 0):
            for s in ti.static(range(Q)):
                neighbor_pos = periodic_index(I+e_xyz[s])
                if (is_solid[neighbor_pos]<=0):
                    stream_f[neighbor_pos][s] = collide_f[I][s]
                else:
                    stream_f[I][reversed_e_index[s]] = collide_f[I][s]  

def export_VTK(n):
    x = np.linspace(0, lx, lx)
    y = np.linspace(0, ly, ly)
    z = np.linspace(0, lz, lz)
    grid_x = np.linspace(0, lx, lx)
    grid_y = np.linspace(0, ly, ly)
    grid_z = np.linspace(0, lz, lz)
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")
    gridToVTK(
        "./LB_SingelPhase_"+str(n),
        grid_x,
        grid_y,
        grid_z,
        pointData={
            "Solid": np.ascontiguousarray(
               is_solid.to_numpy()[0:lx, 0:ly, 0:lz]
            ),
            "rho": np.ascontiguousarray(rho.to_numpy()[0:lx, 0:ly, 0:lz]),
            "velocity": (
                np.ascontiguousarray(v.to_numpy()[0:lx, 0:ly, 0:lz, 0]),
                np.ascontiguousarray(v.to_numpy()[0:lx, 0:ly, 0:lz, 1]),
                np.ascontiguousarray(v.to_numpy()[0:lx, 0:ly, 0:lz, 2]),
            ),
        },
    )

@ti.kernel
def update_vel():
    for I in ti.grouped(v): 
        v[I] = velocity_vec(I)
        
@ti.kernel
def pass_vel():
    for I in ti.grouped(v): 
        target_v[I] = v[I]

def run(max_step=1000,compute_loss=True):
    step = 0
    init_field()
    
    while step < max_step:  
        boundary_condition()
        collision() 
        streaming()
        step +=1 
        if step%500 == 0:
            export_VTK(step//500)
            print("Export {} vtk".format(step//500))
    
    print("Finishing simulation!")
    if compute_loss:
        update_vel()
        pass_vel()

    print("Finishing computing loss!")

@ti.kernel
def compute_loss():  
    for I in ti.grouped(rho): 
        if (I.x < lx and I.y<ly and I.z< lz and is_solid[I]<=0):
            for i in ti.static(range(3)):
                temp_v[I][i]=0.
                for s in ti.static(range(Q)):
                    temp_v[I][i] = temp_v[I][i] + (stream_f[I][s]* e_xyz[s][i])
                temp_v[I][i] = temp_v[I][i]/rho[I] 

            loss[None]+= ((temp_v[I].norm()-target_v[I].norm()))**2/((lx-2)*ly*lz-solid_count)
            #loss[None]+= (temp_v[I].x-target_v[I].x)**2
            if temp_v[I].x>0:
                sum_v[None]+=temp_v[I].x
            if I.x !=0 and I.x!=lx-1:
                sum_rho[None] +=rho[I]
                
##set target velocity field 
dp[None] = 3.0
pressure_gradient = (dPress(dp[None])-dPress(rho0))/lx

target_p = dp[None]
run(max_step=10000)
ti.sync()


1238145 0.3746742424242424
Export 1 vtk
Export 2 vtk
Export 3 vtk


KeyboardInterrupt: 

In [3]:
loss[None] = sum_v[None] =sum_rho[None]=0.
compute_loss()
mean_v =sum_v[None]/((lx-2)*ly*lz-solid_count)
mean_rho = sum_rho[None]/((lx-2)*ly*lz-solid_count)

viscosity_lbm = 0. # Not converted yet 
viscosity_real = 1e-3 # Pa. s # viscosity of water at 20 C (Pascal seconds)

# Unit conversion 1:
lu_to_m = 3e-5 # d50 = 0.68 mm needs 17 lu scale factor is 3 means

def calc_k(mean_v,mean_rho,pressure_gradient):
    kviscosity_lbm = 1/3*(tau-0.5) 
    kviscosity_real = 1e-6 # kinematic viscosity of water at 20 C
    
    # Unit conversion 2:
    mu_to_kg= 1000*lu_to_m**3/mean_rho # 1 mu = ?? kg 
    ts_to_s = kviscosity_lbm/kviscosity_real*lu_to_m**2 # 1 ts =  ?? second
    
    pressure_lbm_to_Pa = mu_to_kg/lu_to_m/(ts_to_s**2)  # 1 pressure_lbm = ?? Pa
    kviscosity_lbm_to_m2Ps = kviscosity_real/kviscosity_lbm # 1 v_lbm = ?? m2/s2 

    # Pair of variables:
    mean_v_lbm = mean_v
    mean_v_real = mean_v*(lu_to_m/ts_to_s)
    cylinder_diameter_lbm = lx/2
    cylinder_diameter_real = cylinder_diameter_lbm*lu_to_m
    pressure_gradient_lbm = pressure_gradient
    pressure_gradient_real = pressure_gradient_lbm*pressure_lbm_to_Pa/lu_to_m

    Reynolds_real = mean_v_real*cylinder_diameter_real/kviscosity_real
    Reynolds_lbm = mean_v_lbm*cylinder_diameter_lbm/kviscosity_lbm

    k_lbm = mean_v_lbm*porosity/pressure_gradient_lbm*kviscosity_lbm*mean_rho
    k_real = mean_v_real*porosity/pressure_gradient_real*kviscosity_real*1000
    
    return Reynolds_real,Reynolds_lbm,k_lbm,k_real,pressure_gradient_real,mean_v_real

print(calc_k(mean_v,mean_rho,pressure_gradient))

from IPython.display import display, Latex
mean_dia = 0.6/1000

display(Latex('${\displaystyle \kappa ={\mathit {\Phi }}_{\mathrm {s} }^{2}{{\epsilon ^{3}D_{\mathrm {p} }^{2}}/({180(1-\epsilon )^{2}}}})$'))
ck_K = porosity**3*(mean_dia)**2/(90*1.4408**2*(1-porosity)**2)
# yang_k = porosity**3*(mean_dia)**2*(porosity-0.2146)/31/(1-porosity)**1.3
print(ck_K)

(4.426212811760722, 4.426212811760722, 0.2531893771561408, 2.2787043944052676e-10, 2425.922285966008, 0.0014754042705869073)


<IPython.core.display.Latex object>

2.591799903921451e-10


In [4]:
# Create a dictionary
data = {'dloss_dp':[],'dp':[],'loss':[],'mean_rho_lbm':[],'mean_v_lbm':[],
       'Reynolds_real':[],'Reynolds_lbm':[],'k_lbm':[],'k_real':[],'pressure_gradient_lbm':[],
        'pressure_gradient_real':[],'mean_v_real':[],}

# initial value of dp
# Initialize pressure gradient and learning rate
dp[None] = 1.2
learning_rate = 0.01

# Initialize gradient and loss
loss.grad[None] = 1
init_loss = 0

# Arrays to store derivatives and computed values
dp_array = []
loss_array = []
mean_vs = []
mean_rhos = []

# Optimization loop
for iteration in range(100):
    # Initialize simulation
    init_field()
    
    # Run simulation for 2000 time steps
    for step in range(10000):
        with ti.ad.Tape(loss):
            # Apply boundary conditions
            boundary_condition()
            # Compute collision
            collision()
            # Stream populations
            streaming()
            # Reset loss and sum values
            loss[None] = sum_v[None] = sum_rho[None] = 0.
            # Compute loss
            compute_loss()
    
    # Store initial loss
    if iteration == 0:
        init_loss = loss[None]
    
    # Store computed values
    data['loss'].append(loss[None])
    data['pressure_gradient_lbm'].append((dPress(dp[None])-dPress(rho0))/(lx-2))
    data['mean_v_lbm'].append(sum_v[None]/((lx-2)*ly*lz-solid_count))
    data['mean_rho_lbm'].append(sum_rho[None]/((lx-2)*ly*lz-solid_count))
    data['dloss_dp'].append(dp.grad[None])
    data['dp'].append(dp[None])
    
    # Print iteration information
    print("dloss/ddp = {}, dp = {}, loss = {} at no. {} iteration.".format(
        dp.grad[None], dp[None], loss[None], iteration
    ))
    
    data['Reynolds_real'],data['Reynolds_lbm'],data['k_lbm'],data['k_real'],    data['pressure_gradient_real'],data['mean_v_real']=    calc_k(np.array(data['mean_v_lbm']),np.array(data['mean_rho_lbm']),np.array(data['pressure_gradient_lbm']))
    
    new = pd.DataFrame.from_dict(data)
    new.to_csv('data_Hamburg.csv',index=False) 
    
    # Update pressure gradient
    dp[None] += dp.grad[None] * learning_rate
    
    # Stop optimization if gradient is small
    if dp.grad[None] < 0.1:
        break

KeyboardInterrupt: 

In [ ]:
data['Reynolds_real'],data['Reynolds_lbm'],data['k_lbm'],data['k_real'],\
data['pressure_gradient_real'],data['mean_v_real']=\
calc_k(np.array(data['mean_v_lbm']),np.array(data['mean_rho_lbm']),np.array(data['pressure_gradient_lbm']))
new = pd.DataFrame.from_dict(data)
new.to_csv('data.csv',index=False)  

def plot_curves(
    x,
    y,
    xlabel=None,
    ylabel=None,
    xlim=None,
    ylim=None,
    labels=None,
    colors=None,
    label_loc=None,
    save_fig=False,
    yscale=None,
    line_style=None,
    marker_style=['',''],
):

    if not labels:
        labels = []
        for i in range(len(x)):
            labels.append(i + 1)

    if line_style is not None:
        if colors:
            for i in range(len(x)):
                plt.plot(
                    x[i],
                    y[i],
                    linestyle=line_style[i],
                    marker=marker_style[i],
                    markersize=3,
                    color=colors[i],
                )
        else:
            for i in range(len(x)):
                plt.plot(x[i], y[i], linestyle=line_style[i], marker="o", markersize=2)
    else:
        if colors:
            for i in range(len(x)):
                plt.plot(
                    x[i], y[i], linestyle="-", marker="o", markersize=2, color=colors[i]
                )
        else:
            for i in range(len(x)):
                plt.plot(x[i], y[i], linestyle="-", marker="o", markersize=2)

    plt.legend(labels, fontsize=10)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel(xlabel, fontsize=12)
    plt.grid(True, color="k", linestyle="--")

    if save_fig:
        plt.savefig("2d_permeability", bbox_inches="tight", dpi=150)


In[ ]:


data = pd.read_csv('data.csv',sep=',')


In[ ]:


plot_curves([data['pressure_gradient_real']],[data['mean_v_real']*1000],xlabel='Pressure Gradient (Pa/m)',
    ylabel='Mean Flow Velocity (mm/s)',
    xlim=None,
    ylim=None,
    labels=[r'$\eta$ = 0.63'],
    colors=['black'],
    label_loc=None,
    save_fig=True,
    yscale=None,
    line_style=['--'],
    marker_style=['o',''],)


In[ ]:


target_loss = 0

final_index = min(range(len(data['loss'])), key=lambda i: abs(data['loss'][i]-target_loss))

init_p_diff = data['pressure_gradient_real'][0]*lx/10000

target_p_diff = data['pressure_gradient_real'][-1]*lx/10000

pressure_diff = data['pressure_gradient_real']*lx/10000
print(init_p_diff,target_p_diff,pressure_diff)


In[ ]:


plt.rcParams['font.family'] = 'DeJavu Serif' 
plt.rcParams["font.serif"] = "Times New Roman"


fig,ax1 = plt.subplots(figsize=(8,5),dpi=120)
# ax1.set_xlim(0.99, 1.61)
# ax1.set_ylim(400, 1200)


ax1.scatter(init_p_diff,init_loss,color='blue',label = 'initial values')

ax1.scatter(target_p_diff,target_loss,color='red',label = 'target values',marker='p', s=80)

ax1.plot(pressure_diff, data['loss'], linestyle='--',marker='o', markersize=5, 
markerfacecolor='none',color='black',label = 'trajectory of differentiation path')
# ax1.set_xlim([0.0,0.09])
# ax1.set_ylim([0, 120])

plt.xlabel(r'pressure difference ($\rm mu·lu^{-1}t^{-2}$)', fontsize=10)
# plt.xlabel(r'pressure difference ($\rm Pa$)', fontsize=10)
plt.ylabel('loss', fontsize=10)

left, bottom, width, height = [0.6, 0.57, 0.28, 0.28]
ax2 = fig.add_axes([left, bottom, width, height])

ax2.scatter(target_p_diff,target_loss,color='red',label = 'target values',marker='p', s=80)

ax2.plot(pressure_diff, data['loss'], linestyle='--',marker='o', markersize=5, 
markerfacecolor='none',color='black',label = 'trajectory of differentiation path')
ax2.set_xlim([0.087, 0.088])
ax2.set_ylim([-0.0, 0.04])

ax1.legend(fontsize=9,loc=(0.05,0.05))
plt.xlabel(r'pressure difference ($\rm mu·lu^{-1}t^{-2}$)', fontsize=9)
# plt.xlabel(r'pressure difference ($\rm Pa$)', fontsize=9)
plt.ylabel('loss', fontsize=9)
plt.show()    